# Desafio Técnico em Python

## Instalando dependências e importando bibliotecas

In [2]:
!pip install pandas==1.5.3 -q
!pip install pandas-gbq==0.17.4 -q
!pip install google-cloud-bigquery==3.16.0 -q
!pip install pyarrow==6.0.0 -q
!pip install basedosdados -q
import basedosdados as bd
import pandas as pd
import plotly.express as px

# Billing project ID (Google Cloud Platform)
# !!! Troque pelo seu project ID
PROJECT_ID = "teste-prefeitura-413816"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 1.5.3 which is incompatible.
xarray 2024.6.0 requires pandas>=2.0, but you have pandas 1.5.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.15.0 requires pyarrow>=8.0.0, but you have pyarrow 7.0.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.4.1 requir

## Paleta de cores

In [3]:
BLUE_HIGHLIGHT = "#3d87ff"
LIGHT_GRAY = "#9b9b9b"

## Requisitando dataframes

In [4]:
query_chamados_01_04 = "SELECT * FROM `datario.adm_central_atendimento_1746.chamado` WHERE DATE(data_inicio) = '2023-04-01'"
query_bairros = "SELECT * FROM `datario.dados_mestres.bairro`"
query_chamados_2022_2023 = "SELECT data_inicio, tipo, subtipo FROM `datario.adm_central_atendimento_1746.chamado` WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'"
query_eventos = "SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`"

In [5]:
# Executa queries e cria DataFrames pandas de todos os resultados

# Chamados realizados no dia 01/04/2023
bd_df_chamados_01_04 = bd.read_sql(query_chamados_01_04, billing_project_id=PROJECT_ID)
df_chamados_01_04 = pd.DataFrame(data = bd_df_chamados_01_04)

# Tabela de bairros
bd_df_bairros = bd.read_sql(query_bairros, billing_project_id=PROJECT_ID)
df_bairros = pd.DataFrame(data = bd_df_bairros)

# Chamados entre 2022 e 2023
bd_df_chamados_2022_2023 = bd.read_sql(query_chamados_2022_2023, billing_project_id=PROJECT_ID)
df_chamados_2022_2023 = pd.DataFrame(data = bd_df_chamados_2022_2023)

# Tabela de eventos
bd_df_eventos = bd.read_sql(query_eventos, billing_project_id=PROJECT_ID)
df_eventos = pd.DataFrame(data = bd_df_eventos)

Downloading: 100%|██████████| 4/4 [00:00<00:00, 21.98rows/s]


## Funções

In [6]:
def plot_bar(data, x, y, color, text, title, subtitle, labels, color_discrete_sequence=None, orientation='v'):
  fig = px.bar(data,
               x = x,
               y = y,
               color = color,
               text = text,
               title=title + f' <br><sup>{subtitle}</sup>',
               labels=labels,
               color_discrete_sequence=color_discrete_sequence)
  fig.update_traces(textposition='outside', showlegend=False)
  fig.update_layout(width=1080, bargap=0.2)
  fig.show()

## Pergunta 1
Quantos chamados foram abertos no dia 01/04/2023?

In [7]:
chamados_04_2023 = df_chamados_2022_2023.loc[df_chamados_2022_2023['data_inicio'].between('2023-04-01', '2023-04-08')]
chamados_04_2023['data_inicio'] = chamados_04_2023['data_inicio'].dt.date

chamados_04_2023_por_dia = chamados_04_2023.groupby('data_inicio')['data_inicio'].count().reset_index(name='chamados')


plot_bar(data=chamados_04_2023_por_dia,
         x='data_inicio',
         y='chamados',
         color='data_inicio',
         text='chamados',
         title='Chamados por dia nos dias 01/04/23 a 07/04/23',
         subtitle='1756 chamados foram abertos no dia 01/04/23',
         labels={'data_inicio': 'Dia', 'chamados': 'Chamados'},
         color_discrete_sequence=[BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY]
         )

## Pergunta 2
Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?


In [8]:
# Agrupa por tipo
df_chamados_por_tipo = df_chamados_01_04.groupby('tipo')['tipo'].count().reset_index(name='chamados')

# Chamados por tipo em ordem decrescente
df_chamados_por_tipo = df_chamados_por_tipo.sort_values(by='chamados', ascending=False)

plot_bar(data=df_chamados_por_tipo.iloc[0:5],
         x='tipo',
         y='chamados',
         color='tipo',
         text='chamados',
         title='Chamados por tipo no dia 01/04/2023',
         subtitle='"Estacionamento Irregular" foi o tipo de Chamado mais aberto neste dia',
         labels={'tipo': 'Tipo de Chamado', 'chamados': 'Chamados'},
         color_discrete_sequence=[BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY]
         )

## Pergunta 3
Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?


In [9]:
# Merge, similar a inner join do SQL, dos DataFrames de Chamados e Bairros
df_merge_chamados_bairros = pd.merge(df_chamados_01_04, df_bairros, on='id_bairro')

# Agrupa os dados por bairro e quantidade de chamados de cada bairro
df_chamados_por_bairro = df_merge_chamados_bairros.groupby('nome')['nome'].count().reset_index(name='chamados')

# Chamados por bairro em ordem decrescente
df_chamados_por_bairro = df_chamados_por_bairro.sort_values(by='chamados', ascending=False)

plot_bar(data=df_chamados_por_bairro.iloc[0:3],
         x='nome',
         y='chamados',
         color='nome',
         text='chamados',
         title='Bairros com os maiores números de chamados no dia 01/04/2023',
         subtitle='Campo grande é o bairro com o maior número de chamados abertos neste dia: 113 chamados',
         labels={'nome': 'Bairro', 'chamados': 'Chamados'},
         color_discrete_sequence=[BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY]
         )

## Pergunta 4
Qual o nome da subprefeitura com mais chamados abertos nesse dia?


In [10]:
# Agrupa os dados por subprefeitura e quantidade de chamados de cada subprefeitura
df_chamados_subprefeitura = df_merge_chamados_bairros.groupby('subprefeitura')['subprefeitura'].count().reset_index(name = 'chamados')

# Chamados por prefeitura em ordem decrescente
df_chamados_subprefeitura_desc = df_chamados_subprefeitura.sort_values(by = "chamados", ascending=False)

plot_bar(data=df_chamados_subprefeitura_desc.iloc[0:5],
         x='subprefeitura',
         y='chamados',
         color='subprefeitura',
         text='chamados',
         title='Subprefeituras com os maiores números de chamados no dia 01/04/2023',
         subtitle='A Zona Norte é a subprefeitura com o maior número de chamados abertos neste dia: 510 chamados',
         labels={'subprefeitura': 'Subprefeitura', 'chamados': 'Chamados'},
         color_discrete_sequence=[BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY]
         )

## Pergunta 5
Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?


In [11]:
# Realiza o LEFT JOIN das tabelas de Chamados e Bairros
df_merge_left_chamados_bairros = pd.merge(df_chamados_01_04, df_bairros, how='left',  on='id_bairro')

# Seleciona os chamados não associados a bairro ou subprefeitura na tabela Bairro
df_merge_chamados_bairros_null = df_merge_left_chamados_bairros[df_merge_left_chamados_bairros['nome'].isnull() | df_merge_left_chamados_bairros['subprefeitura'].isnull()]

# Agrupa por tipo
df_chamados_sem_bairro_por_tipo = df_merge_chamados_bairros_null.groupby('tipo')['tipo'].count().reset_index(name='chamados')

# Chamados por tipo em ordem decrescente
df_chamados_sem_bairro_por_tipo = df_chamados_sem_bairro_por_tipo.sort_values(by='chamados', ascending=False)

plot_bar(data=df_chamados_sem_bairro_por_tipo,
         x='chamados',
         y='tipo',
         color='tipo',
         text='chamados',
         title='Chamados não associados a bairro no dia 01/04/2023 agrupados por Tipo',
         subtitle='A maioria dos chamados neste dia foram do tipo Ônibus',
         labels={'tipo': 'Tipo', 'chamados': 'Chamados'},
         orientation='h'
         )

In [12]:
# Agrupa por subtipo
df_chamados_sem_bairro_por_tipo = df_merge_chamados_bairros_null.groupby('subtipo')['subtipo'].count().reset_index(name='chamados')

# Chamados por subtipo em ordem decrescente
df_chamados_sem_bairro_por_tipo = df_chamados_sem_bairro_por_tipo.sort_values(by='chamados', ascending=False)

plot_bar(data=df_chamados_sem_bairro_por_tipo,
         x='chamados',
         y='subtipo',
         color='subtipo',
         text='chamados',
         title='Chamados não associados a bairro no dia 01/04/2023 agrupador por Subtipo',
         subtitle='A maioria dos chamados do tipo Ônibus foram realizados para Verificação de ar condicionado inoperante',
         labels={'subtipo': 'subtipo', 'chamados': 'Chamados'},
         orientation='h'
         )

### Porque isto acontece?

- Ônibus, BRT e TEC: Ônibus e Unidades de transporte podem não estar associadas a um bairro, ou estar associadas a mais de um bairro, impossibilitando a associação no sistema.
- Atendimento ao cidadão e Alvará: Solicitações de suporte ou orientação realizadas no app móvel no smartphone não estão associadas a uma localidade.
- Defesa do consumidor: Chamados de Verificação de problemas com produtos ou serviços também não necessáriamente estão associados a um bairro ou subprefeitura.

## Pergunta 6
Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [13]:
chamados_sossego = df_chamados_2022_2023.loc[df_chamados_2022_2023['subtipo'] == 'Perturbação do sossego']

print(f"O número de chamados com subtipo 'Perturbação do sossego' abertos desde 01/01/2022 até 31/12/2023 é: {len(chamados_sossego)} chamados")

O número de chamados com subtipo 'Perturbação do sossego' abertos desde 01/01/2022 até 31/12/2023 é: 42830 chamados


## Pergunta 7
Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).


In [14]:
# Função que adiciona a coluna de evento ao df chamados_eventos de acordo com a data

def adiciona_evento(chamados_eventos, df_eventos):
  """
  Percorre um dataframe chamados_eventos linha a linha, analisa o campo data_inicio
  e adiciona o nome do evento correspondente a uma nova coluna 'evento'.

  Args:
    chamados_eventos: DataFrame com os chamados.
    df_eventos: DataFrame com os eventos.

  Returns:
    DataFrame com a nova coluna 'evento'.
  """
  chamados_eventos['evento'] = ''
  for index, row in chamados_eventos.iterrows():
    data_inicio = row['data_inicio']
    evento_encontrado = False
    for _, event_row in df_eventos.iterrows():
      if data_inicio >= event_row['data_inicial'] and data_inicio <= event_row['data_final']:
        chamados_eventos.loc[index, 'evento'] = event_row['evento']
        evento_encontrado = True
        break
    if not evento_encontrado:
      chamados_eventos.loc[index, 'evento'] = 'Sem evento'
  return chamados_eventos

In [15]:
# Adiciona o evento de cada chamado
chamados_sossego_eventos = adiciona_evento(chamados_sossego, df_eventos)

# Filtra apenas os chamados com evento
chamados_sossego_eventos = chamados_sossego_eventos[chamados_sossego_eventos['evento'] != 'Sem evento']

chamados_sossego_eventos

,data_inicio,tipo,subtipo,evento
3691,2022-09-11 08:22:49,Perturbação do sossego,Perturbação do sossego,Rock in Rio
3692,2022-09-11 14:21:57,Perturbação do sossego,Perturbação do sossego,Rock in Rio
3698,2022-09-10 20:50:59,Perturbação do sossego,Perturbação do sossego,Rock in Rio
3700,2022-09-11 13:34:55,Perturbação do sossego,Perturbação do sossego,Rock in Rio
3701,2022-09-11 01:14:32,Perturbação do sossego,Perturbação do sossego,Rock in Rio
...,...,...,...,...
1507891,2023-02-18 19:23:34,Perturbação do sossego,Perturbação do sossego,Carnaval
1507911,2023-02-18 12:02:57,Perturbação do sossego,Perturbação do sossego,Carnaval
1507922,2023-02-18 17:07:39,Perturbação do sossego,Perturbação do sossego,Carnaval
1507923,2023-02-21 17:56:52,Perturbação do sossego,Perturbação do sossego,Carnaval


### Observação
Embora seja possível realizar este JOIN com Python, é muito menos custoso, mais performático e simples fazê-lo diretamente com SQL, conforme o exemplo abaixo:

In [16]:
# Query
query_chamados_sossego_eventos = """
SELECT *
FROM `datario.adm_central_atendimento_1746.chamado` Chamados
INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` Eventos
ON (DATE(Chamados.data_inicio) BETWEEN Eventos.data_inicial AND Eventos.data_final)
WHERE subtipo = "Perturbação do sossego"
"""

bd_df_chamados_sossego_eventos = bd.read_sql(query_chamados_sossego_eventos, billing_project_id=PROJECT_ID)
df_chamados_sossego_eventos = pd.DataFrame(data = bd_df_chamados_sossego_eventos)
df_chamados_sossego_eventos.head()

Downloading: 100%|██████████| 1214/1214 [00:00<00:00, 1674.15rows/s]


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18330346,13,2023-02-19 11:58:16,2023-02-19 16:15:42,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,18333177,11,2023-02-20 23:49:58,2023-02-24 08:17:28,38,2,37515,144,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
2,18330400,11,2023-02-19 12:32:41,2023-02-27 09:02:58,24,2,80788,<NA>,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
3,18329191,11,2023-02-18 18:36:46,2023-02-24 10:35:25,90,3,54205,469,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,1,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
4,18330619,13,2023-02-19 15:03:45,2023-02-27 09:20:48,128,4,232686,81,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554


## Pergunta 8
Quantos chamados desse subtipo foram abertos em cada evento?


In [17]:
# Agrupa chamados por evento
chamados_sossego_por_evento = chamados_sossego_eventos.groupby('evento')['evento'].count().reset_index(name='chamados')

plot_bar(data=chamados_sossego_por_evento,
         x='evento',
         y='chamados',
         color='evento',
         text='chamados',
         title='Chamados de "Perturbação de sossego" criados em cada evento nos anos 2022 e 2023',
         subtitle='O Rock in Rio foi o evento com mais chamados deste subtipo criados nos dois anos',
         labels={'evento': 'Evento', 'chamados': 'Chamados'}
         )

## Pergunta 9
Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [18]:
# Copia DataFrame com chamados do subtipo 'Perturbação do sossego' com a coluna de eventos criado na Pergunta 7
chamados_sossego_eventos_copy = chamados_sossego_eventos.copy()

# Transforma a coluna data_inicio do tipo DataTime para Date, de forma a poder agrupar por data_inicio
chamados_sossego_eventos_copy['data_inicio'] = chamados_sossego_eventos_copy['data_inicio'].dt.date

# Agrupa os chamados por evento e data_inicio
chamados_por_dia_evento = chamados_sossego_eventos_copy.groupby(['evento', 'data_inicio'])['data_inicio'].count().reset_index(name='chamados')

# Calcula a média de chamados por dia para cada evento e arredonda para 2 casas decimais
media_chamados_por_dia = chamados_por_dia_evento.groupby('evento')['chamados'].mean().round(2).reset_index(name='media_chamados_por_dia')

plot_bar(data=media_chamados_por_dia,
         x='evento',
         y='media_chamados_por_dia',
         color='evento',
         text='media_chamados_por_dia',
         title='Eventos com a maior média de chamados de "Perturbação de sossego" por dia',
         subtitle='O Rock in Rio foi o evento com a maior média de chamados por dia deste subtipo entre os três eventos',
         labels={'evento': 'Evento', 'media_chamados_por_dia': 'Média de Chamados por Dia'}
         )

## Pergunta 10
Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [19]:
# Calcula a média diária de chamados abertos para todo o período
chamados_sossego_geral = chamados_sossego.copy()
chamados_sossego_geral['data_inicio'] = chamados_sossego_geral['data_inicio'].dt.date
chamados_sossego_geral = chamados_sossego_geral.groupby('data_inicio')['data_inicio'].count().reset_index(name='chamados')
media_chamados_sossego = chamados_sossego_geral['chamados'].mean().round(2)

# Adiciona a média geral ao dataframe media_chamados_por_dia
nova_linha = pd.DataFrame({'evento': ['Geral'], 'media_chamados_por_dia': [media_chamados_sossego]})
if media_chamados_por_dia[media_chamados_por_dia['evento'] == 'Geral'].empty:
  media_chamados_por_dia = pd.concat([media_chamados_por_dia, nova_linha], ignore_index=True)

In [20]:
media_chamados_por_dia['media_chamados_por_dia']

,media_chamados_por_dia
0,60.25
1,46.33
2,119.14
3,61.98


In [21]:
media_geral = media_chamados_por_dia.loc[media_chamados_por_dia.evento == 'Geral'].media_chamados_por_dia.values[0]

fig = px.bar(media_chamados_por_dia[media_chamados_por_dia['evento'] != 'Geral'],
              x='evento',
         y='media_chamados_por_dia',
         color='evento',
         text='media_chamados_por_dia',
         title='Média Bienal de Chamados de "Perturbação de sossego" por dia por Evento' + f' <br><sup>Rock in Rio 2x acima do valor da Média Bienal ({media_geral})</sup><br><sup>Período: 01/01/2022 - 31/12/2023</sup>',
         labels={'evento': 'Evento', 'media_chamados_por_dia': 'Média de Chamados por Dia'})

fig.update_traces(textposition='outside', showlegend=False)
fig.update_layout(width=1080, bargap=0.2)

# Adiciona linha pontilhada com a média anual geral
fig.add_hline(y = media_geral,
              line_dash="dot",
              line_color="black",
              line_width=3,
              label=dict(
              text="Média Bienal = " + f"{media_geral}",
              textposition="end",
              font=dict(size=12, color="black")
              ))
fig.show()